##### @author: Nikita
##### @date: 13-05-2021 Tuesday
##### @email: nikitarora.21@gmail.com

In [1]:
import re
import json
import codecs
from math import sqrt
from collections import Counter
from collections import OrderedDict

In [2]:
def words(text):
    return re.findall(r'\w+', text.lower())     # \w+ matches one or more word characters (i.e., [a-zA-Z0-9_]).

WORDS = Counter(words(open('big.txt').read()))

# probability of 'word'
def P(word, N=sum(WORDS.values())):
    return WORDS[word] / float(N)

# most probable spelling correction for word
def correction(word): 
    return max(candidates(word), key=P)

# generate possible spelling corrections for word
def candidates(word): 
    return (known([word]) or known(edits1(word)) or known(edits2(word)) or [word])

# the subset of `words` that appear in the dictionary of WORDS
def known(words): 
    return set(w for w in words if w in WORDS)

# all edits that are one edit away from `word`
def edits1(word):
    letters    = 'abcdefghijklmnopqrstuvwxyz'
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
    replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
    inserts    = [L + c + R               for L, R in splits for c in letters]
    return set(deletes + transposes + replaces + inserts)

# all edits that are two edits away from `word`.
def edits2(word):
    return (e2 for e1 in edits1(word) for e2 in edits1(e1))

def saveDict(path):
    with codecs.open(path, mode='w', encoding='UTF8') as dict_file:
        # big_dict = sorted(WORDS.items(), key=lambda x: x[1], reverse=True)
        big_dict = json.dumps(WORDS, ensure_ascii=False)
        dict_file.write(big_dict)
        
# converting word to vector        
def word2vec(word):
    from collections import Counter
    from math import sqrt

    # count the characters in word
    cw = Counter(word)
    # precomputes a set of the different characters
    sw = set(cw)
    # precomputes the "length" of the word vector
    lw = sqrt(sum(c*c for c in cw.values()))

    # return a tuple
    return cw, sw, lw

#formula for finding cosine distance between two vectors
def cosdis(v1, v2):
    # which characters are common to the two words?
    common = v1[1].intersection(v2[1])
    # by definition of cosine distance we have
    return sum(v1[0][ch]*v2[0][ch] for ch in common)/v1[2]/v2[2]

#ranking the candidates for the given word by finding the cosine similarity between the misspelled word and all the generated candidates
def rank(word):
    i=0
    check_similarity = [] 
    check_similarity = candidates(word)
    
    vector_w={}
    for w in check_similarity:
        vector_w[i] = word2vec(w)
        i += 1
    
    j=0
    sim_dict = {}
    vector_in = word2vec(word)
    check_sim = list(check_similarity)
    for v in vector_w:
        dis = cosdis(vector_in, vector_w[v])
        sim_dict[check_sim[j]] = dis
        j+=1
        
    sorted_tuples = sorted(sim_dict.items(), key=lambda item: item[1], reverse=True)
    return sorted_tuples[:3]

In [8]:
def main():
    #print(len(WORDS))
    print(sum(WORDS.values()))
    #print(WORDS.most_common(10))
    #print(max(WORDS, key=P))
    word = input("Enter the word: ")
    if word in WORDS:
        print("You have entered correctly spelled word")
    else:
        print("Oops! May be you want to type...", rank(word))
    
if __name__ == '__main__':
    main()

1115585


Enter the word:  gratful


Oops! May be you want to type... [('grateful', 0.9354143466934852)]


### References

##### https://norvig.com/spell-correct.html
##### https://www.machinelearningplus.com/nlp/cosine-similarity/
##### https://stackoverflow.com/questions/29484529/cosine-similarity-between-two-words-in-a-list
##### https://towardsdatascience.com/spelling-correction-how-to-make-an-accurate-and-fast-corrector-dc6d0bcbba5f